# Azure Arc Enabled SQL Managed Instance in AKS

This notebook and repo will create a
- AKS Cluster
- Custom Location
- Azure Arc Enabled Direct Connected Data Controller
- 3 replica Azure Arc Enabled SQL Managed Instance

It requires

- Azure Cli [install from here](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli) 
- kubectl - I install with chocolatey `choco install kubernetes-cli -y` or [install instructions](https://kubernetes.io/docs/tasks/tools/install-kubectl-windows/)

## First set some variables for the session

I use the SecretsManagement PowerShell module to store my secrets which can be installed with `Install-Module SecretsManagement`. I add secrets with `Set-Secret -Name nameofsecret -Secret secretvalue`.


In [ ]:
$location = 'eastus' # Azure location
$resourceGroup = 'beardarc' # Exisiting resource group

$aksClusterName = 'beard-aks-cluster' #name of the AKS cluster
$clusterNodePoolSize = "Standard_DS4_v2" # VM size of the AKS cluster
$customLocation  = 'beard-aks-cluster-location' # Name of the custom location
$dataController= "ben-aks-direct" # name of the data controller

# I store the user name and password for the data controller with SecretsManagement
$benscreds = New-Object System.Management.Automation.PSCredential ((Get-Secret -Name beardmi-benadmin-user -AsPlainText), (Get-Secret -Name beardmi-benadmin-pwd))
$ENV:AZDATA_USERNAME="$($benscreds.UserName)"
$ENV:AZDATA_PASSWORD="$($benscreds.GetNetworkCredential().Password)"
$ENV:ACCEPT_EULA = 'yes'
$subscription_id = New-Object System.Management.Automation.PSCredential ('subscription-id', (Get-Secret -Name subscription-id)) 
$ENV:SUBSCRIPTION = "$($subscription_id.GetNetworkCredential().Password)"


## Create the AKS Cluster

Using Azure Cli log into Azure and create an AKS

In [ ]:
# az login
az account set -s $ENV:SUBSCRIPTION 
az aks create -g $resourceGroup -n $aksClusterName --node-vm-size $clusterNodePoolSize # --generate-ssh-keys 
az aks get-credentials -g $resourceGroup -n $aksClusterName --overwrite-existing

# Check that the cluster is running

Using kubectl check that the nodes are ready, they should be named `aks-nodepool*`

In [ ]:
kubectl get nodes

# AZ Cli Extensions

ensure that we have the latest versions of the required AZ Cli extensions

Open a seperate PowerShell window and run this - PowerShell Interactive Notebooks dont like this command!!

````
az extension add --name connectedk8s  
az extension add --name k8s-extension  
az extension add --name customlocation  
az extension add --name arcdata  
az extension update --name connectedk8s  
az extension update --name k8s-extension  
az extension update --name customlocation  
az extension update --name arcdata  
````